# **Multiclass Classification using Keras and TensorFlow on Food-101 Dataset**

In [1]:
import tensorflow as tf
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
import os
import random
%matplotlib inline
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S, EfficientNetV2L
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
import cv2

2022-08-08 10:34:22.431901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-08 10:34:22.732866: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-08-08 10:34:24.069755: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-08-08 10:34:24.069903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_

In [2]:
# !apt-get update
# !apt-get install ffmpeg libsm6 libxext6  -y

In [3]:
# pip install opencv-python

### **Download the Data**

In [2]:
# # Check if GPU is enabled
# print(tf.__version__)
# print(tf.test.gpu_device_name())

In [3]:
# %cd /kaggle/input/food-101/

In [2]:
# # Helper function to download data and extract
# def get_data_extract():
#   if "food-101" in os.listdir():
#     print("Dataset already exists")
#   else:
#     print("Downloading the data...")
#     !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
#     print("Dataset downloaded!")
#     print("Extracting data..")
#     !tar xzvf food-101.tar.gz
#     print("Extraction done!")

In [4]:
# Download data and extract it to folder
# Uncomment this below line if you are on Colab
# get_data_extract()

### **Split the image data into train and test using train.txt and test.txt**

In [5]:
# # Helper method to split dataset into train and test folders
# def prepare_data(filepath, src,dest):
#   classes_images = defaultdict(list)
#   with open(filepath, 'r') as txt:
#       paths = [read.strip() for read in txt.readlines()]
#       for p in paths:
#         food = p.split('/')
#         classes_images[food[0]].append(food[1] + '.jpg')

#   for food in classes_images.keys():
#     print("\nCopying images into ",food)
#     if not os.path.exists(os.path.join(dest,food)):
#       os.makedirs(os.path.join(dest,food))
#     for i in classes_images[food]:
#       copy(os.path.join(src,food,i), os.path.join(dest,food,i))
#   print("Copying Done!")

In [6]:
# %cd food-101/

In [7]:
# prepare_data('/content/food-101/meta/train.txt', '/content/food-101/images', 'train')

In [8]:
# prepare_data('/content/food-101/meta/test.txt', '/content/food-101/images', 'test')

In [9]:
# # Check how many files are in the train folder
# print("Total number of samples in train folder")
# !find train -type d -or -type f -printf '.' | wc -c

Total number of samples in train folder
find: ‘train’: No such file or directory
0


In [15]:
# # Check how many files are in the test folder
# print("Total number of samples in test folder")
# !find test -type d -or -type f -printf '.' | wc -c

Total number of samples in test folder
25250


### **Create a subset of data with few classes(3) - train_mini and test_mini for experimenting**

In [18]:
# # Helper method to create train_mini and test_mini data samples
# def dataset_mini(food_list, src, dest):
#   if os.path.exists(dest):
#     rmtree(dest) # removing dataset_mini(if it already exists) folders so that we will have only the classes that we want
#   os.makedirs(dest)
#   for food_item in food_list :
#     print("Copying images into",food_item)
#     copytree(os.path.join(src,food_item), os.path.join(dest,food_item))
      

In [19]:
# # picking 3 food items and generating separate data folders for the same
# food_list = ['apple_pie','pizza','omelette']
# src_train = 'train'
# dest_train = 'train_mini'
# src_test = 'test'
# dest_test = 'test_mini'

In [10]:
# print("Creating train data folder with new classes")
# dataset_mini(food_list, src_train, dest_train)

In [21]:
# print("Total number of samples in train folder")

# !find train_mini -type d -or -type f -printf '.' | wc -c

Total number of samples in train folder
2250


In [22]:
# print("Creating test data folder with new classes")
# dataset_mini(food_list, src_test, dest_test)

Creating test data folder with new classes
Copying images into apple_pie
Copying images into pizza
Copying images into omelette


In [11]:
# print("Total number of samples in test folder")
# !find test_mini -type d -or -type f -printf '.' | wc -c

### **Data Augmentation and Acc/Loss Visualization**

In [16]:
%cd .. # Run it two times to go to workspace dir

/workspace


In [17]:
K.clear_session()
n_classes = 101
img_width, img_height = 224, 224

train_data_dir = 'persistent/food-101/train'
validation_data_dir = 'persistent/food-101/test'
# train_data_dir = '/Users/dim__gag/python/food-101/data_mini/train_mini'
# validation_data_dir = '/Users/dim__gag/python/food-101/data_mini/test_mini'

nb_train_samples = 2250 #75750
nb_validation_samples = 750 #25250
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')



Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [18]:
# Plot Accuracy and Loss of the model
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()



def plot_Acc_and_Loss(history,title):
    # plot model accuracy
    plt.figure(figsize=(16,8))
    plt.subplot(1,2,1)
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    # plot model loss
    plt.subplot(1,2,2)
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()

## **EfficientNetV2S & SAM**

In [19]:
K.clear_session()
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S, EfficientNetV2L
# base_EffNet = EfficientNetV2S(include_top=False, weights='imagenet')
base_EffNet = EfficientNetV2L(include_top=False, weights='imagenet')

# base_EffNet = EfficientNetV2S(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
#     include_preprocessing=True)


base_EffNet.trainable = False

2022-08-08 10:38:02.736083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-08-08 10:38:02.736162: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-08 10:38:02.736600: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-gpu-5-gwdmm): /proc/driver/nvidia/version does not exist
2022-08-08 10:38:02.737505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FM

473176280/473176280 [==============================] - 7s 0us/step


In [6]:
# base_EffNet.summary()

In [20]:
# Fine tune the model
x = base_EffNet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(3, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
EffNet = Model(inputs=base_EffNet.input, outputs=predictions)

In [8]:
# EffNet.summary()

In [21]:
# <-------------------------------------------------------------------------------------------------------------->
class SAMModel(tf.keras.Model):
    def __init__(self, model, rho=0.05):
        """
        p, q = 2 for optimal results as suggested in the paper
        (Section 2)
        """
        super(SAMModel, self).__init__()
        self.model = model
        self.rho = rho

    def train_step(self, data):
        (images, labels) = data
        e_ws = []
        with tf.GradientTape() as tape:
            predictions = self.model(images)
            loss = self.compiled_loss(labels, predictions)
        trainable_params = self.model.trainable_variables
        gradients = tape.gradient(loss, trainable_params)
        grad_norm = self._grad_norm(gradients)
        scale = self.rho / (grad_norm + 1e-12)

        for (grad, param) in zip(gradients, trainable_params):
            e_w = grad * scale
            param.assign_add(e_w)
            e_ws.append(e_w)

        with tf.GradientTape() as tape:
            predictions = self.model(images)
            loss = self.compiled_loss(labels, predictions)    
        
        sam_gradients = tape.gradient(loss, trainable_params)
        for (param, e_w) in zip(trainable_params, e_ws):
            param.assign_sub(e_w)
        
        self.optimizer.apply_gradients(
            zip(sam_gradients, trainable_params))
        
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        (images, labels) = data
        predictions = self.model(images, training=False)
        loss = self.compiled_loss(labels, predictions)
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def _grad_norm(self, gradients):
        norm = tf.norm(
            tf.stack([
                tf.norm(grad) for grad in gradients if grad is not None
            ])
        )
        return norm

In [22]:
model = SAMModel(EffNet)

In [23]:
# GET THE SAM MODEL
model.trainable = True # Unfreeze the model
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
# checkpointer = ModelCheckpoint(filepath='EfficientSAM.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('EfficientSAM.log')

In [24]:
# model.summary()

In [25]:
# Train the model
K.clear_session()
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, validation_data=validation_generator, validation_steps=nb_validation_samples // batch_size, epochs=400, verbose=1, callbacks=[csv_logger])
# model.save('EfficientSAM.hdf5')

Epoch 1/400


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 596, in run_forever
      self._run_o

In [ ]:
plot_Acc_and_Loss(history, "EfficientNetV2S + SAM")

Try to reload the model, freeze the base model and then unfreeze the last layers of the model.